## Sparse state networks 
Second-order dynamics on a physical network can be described by first-order dynamics on a second-order state network.

We can represent this second-order network by it's _state transition matrix_ $P_{ij}$ with the probabilities for the random walker to transition from state node $i$ to state node $j$.

In this view, we may note that some rows have similar probability distributions. We can measure how similar two probability distributions are with the [Jensen-Shannon Distance](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence).

The idea behind sparse state networks is that we can lump state nodes (within each physical node) that constrain the network flow in a similar way without loosing (much) information.

### Transforms to a general machine learning problem
We will here solve the problem using standard [clustering algorithms](http://scikit-learn.org/stable/modules/clustering.html#clustering) from the [scikit-learn](http://scikit-learn.org/) package.

In order to do that, we have to transform the state network into features usable for machine learning. We can do this with the help of the code in [state_lumping_network.py](./state_lumping_network.py).

In [1]:
from state_lumping_network import StateNetwork

In [2]:
net = StateNetwork()
net.readFromFile("../data/toy_states.net")

Read state network from file '../data/toy_states.net'...
 -> StateNetwork (5 physical nodes, 12 state nodes and 32 links)


![toy_states](../figures/toy_states_full.png)

In [3]:
X, rowToStateId = net.getFeatureMatrix(1)
print("Feature matrix for the central physical node: \n{}\n rowToStateId: {}".format(X, rowToStateId))

Feature matrix for the central physical node: 
[[0.4 0.4 0.1 0.1]
 [0.4 0.4 0.1 0.1]
 [0.1 0.1 0.4 0.4]
 [0.1 0.1 0.4 0.4]]
 rowToStateId: {0: 1, 1: 2, 2: 7, 3: 8}


The `getFeatureMatrix` method removes all-zero rows and columns in the feature matrix but provides a mapping back to the original state network.

### Cluster with scikit-learn
Now we can compare rows pairwise and cluster the most similar rows together. The Jensen-Shannon distance is unfortunately not implemented in scikit-learn (though it exist in a [pull request](https://github.com/scikit-learn/scikit-learn/pull/4191)), but we can use for example `cosine` instead with similar result.

In [4]:
from sklearn import cluster

In [5]:
model = cluster.AgglomerativeClustering(
    linkage="complete",
    # affinity=jensen_shannon_distances,
    affinity="cosine",
    n_clusters=2
)

labels = model.fit_predict(X)
print("Cluster labels in feature matrix space: {}\nCluster labels in state node space: {}".format(
    labels,
    {rowToStateId[i]:clusterId for i,clusterId in enumerate(labels)}
))

Cluster labels in feature matrix space: [1 1 0 0]
Cluster labels in state node space: {1: 1, 2: 1, 7: 0, 8: 0}


### Lump whole network
We can provide a custom clustering algorithm to `StateNetwork`, where we get the feature matrix as input and should provide the cluster labels of that matrix back. With the clustering method above, we need to set the number of clusters, which we for this toy network can be set to half of the number of state nodes.

In [6]:
def getFeatureClusterFunction(clusterRate=0.5):
    def calcClusters(X):
        numStates, numFeatures = X.shape
        if numStates < 2 or numFeatures < 2:
            # Don't cluster if too small
            return list(range(numStates))

        # Can be an adaptive number of clusters based on entropy reduction
        n_clusters = max(1, int(clusterRate * numStates))
        model = cluster.AgglomerativeClustering(
            linkage="complete",
            # affinity=jensen_shannon_distances,
            affinity="cosine",
            n_clusters=n_clusters
        )

        labels = model.fit_predict(X)
        return labels
    return calcClusters

In [7]:
net.clusterStateNodes(clusterFeatureMatrix=getFeatureClusterFunction())

Cluster state nodes...
Generate lumped state network from clustering...
 -> 6 state nodes and 16 links in lumped network.


In [8]:
h1 = net.calcEntropyRate()
h2 = net.calcLumpedEntropyRate()
print("Entropy rate before: {}, after: {}".format(h1, h2))

Entropy rate before: 1.2406426982957872, after: 1.2406426982957874


In [9]:
from pathlib import Path
net.writeLumpedStateNetwork("../output/toy_lumped.net")
print(Path('../output/toy_lumped.net').read_text())

Writing lumped state network to file '../output/toy_lumped.net'...
# physical nodes: 5
# state nodes: 12
# lumped state nodes: 6
*Vertices
1 "1"
2 "2"
3 "3"
4 "4"
5 "5"
*States
#lumpedStateId physicalId lumpedStateIds
1 1 "[7, 8]"
2 1 "[1, 2]"
3 2 "[3, 4]"
4 3 "[5, 6]"
5 4 "[9, 10]"
6 5 "[11, 12]"
*Links
1 5 1.6
1 6 1.6
1 3 0.4
1 4 0.4
2 3 1.6
2 4 1.6
2 5 0.4
2 6 0.4
3 2 2.0
3 4 2.0
4 2 2.0
4 3 2.0
5 1 2.0
5 6 2.0
6 1 2.0
6 5 2.0



### Now we have generated the sparse network (with lossless compression)
![toy_states](../figures/toy_states_full.png)
![toy_states](../figures/toy_states_sparse.png)

To the left, the original second-order network. To the right, the sparse network formed by lumping similar state nodes within each physical node.

### Test with Infomap

In [10]:
import infomap

In [17]:
def partition(inputFilename):
    im = infomap.Infomap("--directed")
    im.network().readInputData(inputFilename)
    im.run()
    print("{}:".format(inputFilename))
    print("{} top modules with codelength {}".format(im.numTopModules(), im.codelength()))
    print("\nState level modules")
    print("#stateId physicalId moduleIndex flow")
    for node in im.iterTree():
        if node.isLeaf():
            print("{} {} {} {}".format(node.stateId, node.physicalId, node.moduleIndex(), node.data.flow))
    print("\nPhysical level modules")
    print("#physicalId moduleIndex flow")
    for node in im.iterTreePhysical():
        if node.isLeaf():
            print("{} {} {}".format(node.physicalId, node.moduleIndex(), node.data.flow))

partition("../data/toy_states.net")
print("\n--------------\n")
partition("../output/toy_lumped.net")


../data/toy_states.net:
2 top modules with codelength 3.0114052384459713

State level modules
#stateId physicalId moduleIndex flow
1 1 0 0.08333333333333333
2 1 0 0.08333333333333333
3 2 0 0.08333333333333333
4 2 0 0.08333333333333333
5 3 0 0.08333333333333333
6 3 0 0.08333333333333333
7 1 1 0.08333333333333334
8 1 1 0.08333333333333334
9 4 1 0.08333333333333334
10 4 1 0.08333333333333334
11 5 1 0.08333333333333334
12 5 1 0.08333333333333334

Physical level modules
#physicalId moduleIndex flow
1 0 0.16666666666666666
2 0 0.16666666666666666
3 0 0.16666666666666666
1 1 0.16666666666666669
4 1 0.16666666666666669
5 1 0.16666666666666669

--------------

../output/toy_lumped.net:
2 top modules with codelength 2.011405238445971

State level modules
#stateId physicalId moduleIndex flow
1 1 0 0.16666666666666669
5 4 0 0.16666666666666669
6 5 0 0.16666666666666669
2 1 1 0.16666666666666669
3 2 1 0.16666666666666669
4 3 1 0.16666666666666669

Physical level modules
#physicalId moduleIndex flow